In [2]:
from model_builder import model_builder
from data_gen_ext import simulate_private_equity_cashflows_ext
import numpy as np

In [48]:
cashflows = simulate_private_equity_cashflows_ext()

In [49]:
# Normalize the cashflows of each fund in the market

def fund_weights(cashflows):
    funds = cashflows['FundID'].unique() 
    weights = {}
    for fund in funds:
        fund_cashflows = cashflows[cashflows['FundID'] == fund]
        weights[fund] = -fund_cashflows['Cashflow'].sum()
    return weights

cashflows_contr = cashflows[cashflows['Cashflow'] < 0]
eq_cashflows_contr = fund_weights(cashflows_contr)

def eq_calc(row):
    fund = row['FundID']
    cf = row['Cashflow']
    
    return cf / eq_cashflows_contr[fund]

# Apply the function row-wise
cashflows['eq_cashflow'] = cashflows.apply(eq_calc, axis=1)

# Drop and rename
cashflows = cashflows.drop(columns=['Cashflow']).rename(columns={'eq_cashflow': 'Cashflow'})

cashflows

,FundID,VintageYear,Strategy,Geography,FundQuality,Quarter,date,Cashflow
0,0,1995,Venture,Europe,Bad,0,1995-03-31,-0.117893
1,0,1995,Venture,Europe,Bad,1,1995-06-30,-0.075568
2,0,1995,Venture,Europe,Bad,2,1995-09-30,-0.068922
3,0,1995,Venture,Europe,Bad,3,1995-12-31,-0.078510
4,0,1995,Venture,Europe,Bad,4,1996-03-31,-0.060517
...,...,...,...,...,...,...,...,...
118945,2399,1994,Buyout,Europe,Bad,52,2007-03-31,0.070475
118946,2399,1994,Buyout,Europe,Bad,53,2007-06-30,0.000000
118947,2399,1994,Buyout,Europe,Bad,54,2007-09-30,0.000000
118948,2399,1994,Buyout,Europe,Bad,55,2007-12-31,0.043193


In [50]:
def portfolio_cashflows(cashflows, portfolio_size):
    # Randomly select fund IDs
    selected_funds = np.random.choice(cashflows['FundID'].unique(), size=portfolio_size, replace=False)
    
    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in selected_funds}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(selected_funds)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [51]:
port_cfs = portfolio_cashflows(cashflows, 200)
port_cfs = port_cfs.copy()
model = model_builder(cashflows, port_cfs)
model

({'Portfolio MOIC': 2.368349985546316,
  'Market MOIC': 2.20305521722451,
  'Timing Alpha MOIC': 0.0006498939074095489,
  'Strategy Alpha MOIC': 0.023361543365379767,
  'Geography Alpha MOIC': 0.058546630982378556,
  'Sizing Alpha MOIC': 0.04216217451828319,
  'Residual Alpha MOIC': 0.040574525548354945},
 {'Portfolio IRR': 0.14710087522587775,
  'Market IRR': 0.13530799444845143,
  'Timing Alpha IRR': -0.00015830380965786706,
  'Strategy Alpha IRR': 0.0021908140882262728,
  'Geography Alpha IRR': 0.008008507095438488,
  'Sizing Alpha IRR': 0.0007805828817975535,
  'Residual Alpha IRR': 0.0009712805216218756})

### Monte-Carlo simulation

In [52]:
market_model = model_builder(cashflows, cashflows)
market_model

({'Portfolio MOIC': 2.20305521722451,
  'Market MOIC': 2.20305521722451,
  'Timing Alpha MOIC': -4.440892098500626e-16,
  'Strategy Alpha MOIC': -4.440892098500626e-16,
  'Geography Alpha MOIC': -4.440892098500626e-16,
  'Sizing Alpha MOIC': 0.0,
  'Residual Alpha MOIC': 1.3322676295501878e-15},
 {'Portfolio IRR': 0.13530799444845143,
  'Market IRR': 0.13530799444845143,
  'Timing Alpha IRR': -8.326672684688674e-17,
  'Strategy Alpha IRR': -8.326672684688674e-17,
  'Geography Alpha IRR': 1.1102230246251565e-16,
  'Sizing Alpha IRR': -2.7755575615628914e-17,
  'Residual Alpha IRR': 8.326672684688674e-17})

In [63]:
number_of_simulations = 15
fund_size = 200
moic_results = []
irr_results = []

for i in range(number_of_simulations):
    port_cfs = portfolio_cashflows(cashflows, fund_size)
    port_cfs = port_cfs.copy()
    model = model_builder(cashflows, port_cfs)
    moic_results.append(model[0])
    irr_results.append(model[1])

In [64]:
def average_dicts(dict_list):
    """Compute the average of a list of dictionaries."""
    keys = dict_list[0].keys()
    avg_dict = {}
    for key in keys:
        avg_dict[key] = sum(d[key] for d in dict_list) / len(dict_list)
    return avg_dict

In [65]:
avg_moic_decomposition = average_dicts(moic_results)
avg_irr_decomposition = average_dicts(irr_results)

In [66]:
avg_irr_decomposition

{'Portfolio IRR': 0.13967777195374761,
 'Market IRR': 0.1353079944484514,
 'Timing Alpha IRR': 0.002192321011313969,
 'Strategy Alpha IRR': 0.00013698296822062066,
 'Geography Alpha IRR': 0.001285640138101307,
 'Sizing Alpha IRR': 0.000138304696771524,
 'Residual Alpha IRR': 0.0006165286908887883}

In [67]:
avg_moic_decomposition

{'Portfolio MOIC': 2.2252519623142093,
 'Market MOIC': 2.20305521722451,
 'Timing Alpha MOIC': 0.007320658710469038,
 'Strategy Alpha MOIC': 0.00564172524386913,
 'Geography Alpha MOIC': 0.012781269047091648,
 'Sizing Alpha MOIC': -0.0075351899963712835,
 'Residual Alpha MOIC': 0.003988282084640845}

## Analyze the individual effects

### strategy effect

In [24]:
venture_ids = cashflows[cashflows['Strategy'] == 'Venture']['FundID'].unique()
buyout_ids = cashflows[cashflows['Strategy'] == 'Buyout']['FundID'].unique()
growth_ids = cashflows[cashflows['Strategy'] == 'Growth']['FundID'].unique()
venture_ids = np.append(venture_ids,(buyout_ids[0], growth_ids[0])) # we need at least one venture fund to make the model work
buyout_ids = np.append(buyout_ids,(venture_ids[0], growth_ids[0])) # we need at least one buyout fund to make the model work
growth_ids = np.append(growth_ids,(venture_ids[0], buyout_ids[0])) # we need at least one growth fund to make the model work

In [25]:
def portfolio_cashflows_by_id(cashflows, ids):

    # Create a random size adjustment for each selected fund
    size_adjustments = {fund: np.random.uniform(0.5, 2.0) for fund in ids}
    
    # Filter the portfolio
    portfolio = cashflows[cashflows['FundID'].isin(ids)].copy()
    
    # Apply the size adjustment
    portfolio['Cashflow'] = portfolio.apply(lambda row: row['Cashflow'] * size_adjustments[row['FundID']], axis=1)
    
    return portfolio

In [26]:
port_cfs_venture = portfolio_cashflows_by_id(cashflows, venture_ids)
port_cfs_venture = port_cfs_venture.copy()
port_cfs_buyout = portfolio_cashflows_by_id(cashflows, buyout_ids)
port_cfs_buyout = port_cfs_buyout.copy()
port_cfs_growth = portfolio_cashflows_by_id(cashflows, growth_ids)
port_cfs_growth = port_cfs_growth.copy()

In [27]:
model_venture = model_builder(cashflows, port_cfs_venture)
model_venture

({'Portfolio MOIC': 2.528864121082101,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': 0.00022781697474272278,
  'Strategy Alpha MOIC': 0.34322553567369196,
  'Geography Alpha MOIC': -0.0022564436823397926,
  'Sizing Alpha MOIC': -0.04250913738661444,
  'Residual Alpha MOIC': -0.008286057732720842},
 {'Portfolio IRR': 0.1674890251827219,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.0005030394450960263,
  'Strategy Alpha IRR': 0.028972995038668126,
  'Geography Alpha IRR': 0.0007343651135306717,
  'Sizing Alpha IRR': -0.0010327312319083626,
  'Residual Alpha IRR': -0.001571739614326495})

In [28]:
model_buyout = model_builder(cashflows, port_cfs_buyout)
model_buyout

({'Portfolio MOIC': 1.8932745042913788,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.0004152628889313448,
  'Strategy Alpha MOIC': -0.3566587636148473,
  'Geography Alpha MOIC': -0.0002367727203451686,
  'Sizing Alpha MOIC': 0.01630609833361718,
  'Residual Alpha MOIC': -0.004183202053455792},
 {'Portfolio IRR': 0.10783871228839584,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': 9.644443888659482e-05,
  'Strategy Alpha IRR': -0.033975104283314614,
  'Geography Alpha IRR': -0.0013425376853895554,
  'Sizing Alpha IRR': 0.0014135295197639092,
  'Residual Alpha IRR': 0.0007572049765955102})

In [30]:
model_growth = model_builder(cashflows, port_cfs_growth)
model_growth

({'Portfolio MOIC': 2.250547420305834,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.003730923076746606,
  'Strategy Alpha MOIC': 0.016201989818888318,
  'Geography Alpha MOIC': 0.016619595828721856,
  'Sizing Alpha MOIC': 0.0029050738106741747,
  'Residual Alpha MOIC': -0.01991072331104471},
 {'Portfolio IRR': 0.14456192922430958,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.0004941532464196596,
  'Strategy Alpha IRR': 0.002215031897355696,
  'Geography Alpha IRR': 0.0019375300465210021,
  'Sizing Alpha IRR': 0.0022835243739518785,
  'Residual Alpha IRR': -0.0022691791689533336})

### Geo effect

In [31]:
america_ids = cashflows[cashflows['Geography'] == 'North America']['FundID'].unique()
eu_ids = cashflows[cashflows['Geography'] == 'Europe']['FundID'].unique()
asia_ids = cashflows[cashflows['Geography'] == 'Asia']['FundID'].unique()
america_ids = np.append(america_ids,(eu_ids[0], asia_ids[0])) # we need at least one venture fund to make the model work
eu_ids = np.append(eu_ids,(america_ids[0], asia_ids[0])) # we need at least one buyout fund to make the model work
asia_ids = np.append(asia_ids,(america_ids[0], eu_ids[0])) # we need at least one growth fund to make the model work

In [32]:
port_cfs_america = portfolio_cashflows_by_id(cashflows, america_ids)
port_cfs_america = port_cfs_america.copy()
port_cfs_eu = portfolio_cashflows_by_id(cashflows, eu_ids)
port_cfs_eu = port_cfs_eu.copy()
port_cfs_asia = portfolio_cashflows_by_id(cashflows, asia_ids)
port_cfs_asia = port_cfs_asia.copy()

In [33]:
model_america = model_builder(cashflows, port_cfs_america)
model_america

({'Portfolio MOIC': 2.6396208881454695,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': 0.022553806183613023,
  'Strategy Alpha MOIC': 0.010950519104901524,
  'Geography Alpha MOIC': 0.368692965794029,
  'Sizing Alpha MOIC': 0.0077969275576905694,
  'Residual Alpha MOIC': -0.00883573773010582},
 {'Portfolio IRR': 0.17022812987785363,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': 0.0017877771722839553,
  'Strategy Alpha IRR': 0.0016159628853012198,
  'Geography Alpha IRR': 0.02734135898415435,
  'Sizing Alpha IRR': 0.0014304191141819855,
  'Residual Alpha IRR': -0.0028365635999218763})

In [34]:
model_eu = model_builder(cashflows, port_cfs_eu)
model_eu

({'Portfolio MOIC': 2.4159776687087815,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.0009422846295557008,
  'Strategy Alpha MOIC': -0.0014420719278276373,
  'Geography Alpha MOIC': 0.17262321481235343,
  'Sizing Alpha MOIC': 0.013093599104776033,
  'Residual Alpha MOIC': -0.005817195886305804},
 {'Portfolio IRR': 0.16148393893724658,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -3.2419462365984586e-05,
  'Strategy Alpha IRR': 0.0007392177381027598,
  'Geography Alpha IRR': 0.01948575436362504,
  'Sizing Alpha IRR': 0.0018824880112515985,
  'Residual Alpha IRR': -0.0014802770352208283})

In [35]:
model_asia = model_builder(cashflows, port_cfs_asia)
model_asia

({'Portfolio MOIC': 1.6595094852994978,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.023160646568719745,
  'Strategy Alpha MOIC': -0.02317502228431323,
  'Geography Alpha MOIC': -0.5596177506659177,
  'Sizing Alpha MOIC': -0.009589904764140345,
  'Residual Alpha MOIC': 0.03659040234724764},
 {'Portfolio IRR': 0.08595735082362668,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.0017419499020160656,
  'Strategy Alpha IRR': -0.002949389898671051,
  'Geography Alpha IRR': -0.052248176063407814,
  'Sizing Alpha IRR': -0.0016894271211575634,
  'Residual Alpha IRR': 0.0036971184870251833})

### Fund quality

In [36]:
good_ids = cashflows[cashflows['FundQuality'] == 'Good']['FundID'].unique()
bad_ids = cashflows[cashflows['FundQuality'] == 'Bad']['FundID'].unique()
good_ids = np.append(good_ids,bad_ids[0]) # we need at least one venture fund to make the model work
bad_ids = np.append(bad_ids,good_ids[0]) # we need at least one buyout fund to make the model work

In [37]:
port_cfs_good = portfolio_cashflows_by_id(cashflows, good_ids)
port_cfs_good = port_cfs_good.copy()    
port_cfs_bad = portfolio_cashflows_by_id(cashflows, bad_ids)
port_cfs_bad = port_cfs_bad.copy()

In [38]:
model_good = model_builder(cashflows, port_cfs_good)
model_good

({'Portfolio MOIC': 2.963707594067612,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': 0.0037403174076477974,
  'Strategy Alpha MOIC': 0.012488456861852626,
  'Geography Alpha MOIC': 0.0120526474932241,
  'Sizing Alpha MOIC': 0.0015763515336675304,
  'Residual Alpha MOIC': 0.6953874135358786},
 {'Portfolio IRR': 0.1981588060277388,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': 0.0007184733930143683,
  'Strategy Alpha IRR': 0.0010990495998028371,
  'Geography Alpha IRR': 0.0005558753593686294,
  'Sizing Alpha IRR': -3.850548064013304e-05,
  'Residual Alpha IRR': 0.05493473783433911})

In [39]:
model_bad = model_builder(cashflows, port_cfs_bad)
model_bad

({'Portfolio MOIC': 1.541586884450072,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.0056064721734148115,
  'Strategy Alpha MOIC': -0.007845604096410774,
  'Geography Alpha MOIC': -0.0035993949528125846,
  'Sizing Alpha MOIC': 0.003069089554867288,
  'Residual Alpha MOIC': -0.6828931411174983},
 {'Portfolio IRR': 0.07082198288690467,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.0005822716148550233,
  'Strategy Alpha IRR': -0.0005440242572734566,
  'Geography Alpha IRR': 0.00045248933609678965,
  'Sizing Alpha IRR': 0.00045663114362046175,
  'Residual Alpha IRR': -0.0698500170425381})

### Vintage effect

In [40]:
vintage_ids = {}
for vint in cashflows['VintageYear'].unique():
    vint_ids = cashflows[cashflows['VintageYear'] == vint]['FundID'].unique()
    vintage_ids[vint] = vint_ids

In [41]:
import random
def pick_one_per_vintage(fund_dict):
    selected_ids = []

    for vintage, fund_ids in fund_dict.items():
        # Randomly pick one fund from this vintage
        fund_ids = list(fund_ids)  # ensure it's a list
        selected_id = random.choice(fund_ids)
        selected_ids.append(selected_id)

    return np.array(selected_ids)

one_from_each = pick_one_per_vintage(vintage_ids)

In [42]:
good_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 1]['FundID'].unique()
bad_vintage_ids = cashflows[cashflows['VintageYear'] % 2 == 0]['FundID'].unique()
good_vintage_ids = np.append(good_vintage_ids,one_from_each) # we need at least one venture fund to make the model work
bad_vintage_ids = np.append(bad_vintage_ids,one_from_each) # we need at least one buyout fund to make the model work

In [43]:
port_cfs_good_vintage = portfolio_cashflows_by_id(cashflows, good_vintage_ids)
port_cfs_good_vintage = port_cfs_good_vintage.copy()
port_cfs_bad_vintage = portfolio_cashflows_by_id(cashflows, bad_vintage_ids)
port_cfs_bad_vintage = port_cfs_bad_vintage.copy()

In [44]:
model_good_vintage = model_builder(cashflows, port_cfs_good_vintage)
model_good_vintage

({'Portfolio MOIC': 2.5391525268540347,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': 0.3101218181352121,
  'Strategy Alpha MOIC': 0.0005587543279950147,
  'Geography Alpha MOIC': -0.010671429353007689,
  'Sizing Alpha MOIC': -0.008787322907759965,
  'Residual Alpha MOIC': 0.009468299416254045},
 {'Portfolio IRR': 0.16718252643789477,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': 0.026856832157967442,
  'Strategy Alpha IRR': -0.0028248882741917503,
  'Geography Alpha IRR': -0.0039821055926791415,
  'Sizing Alpha IRR': -0.0005004487328571783,
  'Residual Alpha IRR': 0.006743961557801409})

In [45]:
model_bad_vintage = model_builder(cashflows, port_cfs_bad_vintage)
model_bad_vintage

({'Portfolio MOIC': 1.912453178799579,
  'Market MOIC': 2.238462407235341,
  'Timing Alpha MOIC': -0.3269468866019516,
  'Strategy Alpha MOIC': -0.040376136416460096,
  'Geography Alpha MOIC': 0.052970839432698646,
  'Sizing Alpha MOIC': 0.0001589659647140973,
  'Residual Alpha MOIC': -0.011816010814763134},
 {'Portfolio IRR': 0.11190288665763065,
  'Market IRR': 0.140889175321854,
  'Timing Alpha IRR': -0.028555901752591054,
  'Strategy Alpha IRR': -0.0071366844418671305,
  'Geography Alpha IRR': 0.0035140878355821026,
  'Sizing Alpha IRR': -0.0004233186167931918,
  'Residual Alpha IRR': 0.0036155283114459313})